In [1]:
%cd ../

/u02/thanhhm/research-idea


In [2]:
import time

from plasma.parallel_processing import communicators, queues
from plasma.parallel_processing.communicators import TreeFlow
from plasma.functional import partials, AutoPipe

/u02/thanhhm/research-idea/.env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute(x):
    time.sleep(2)
    return str(x)

def compute2(x):
    time.sleep(2)
    return x + 'c'

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, tmp):
        return tmp + 'd'

In [4]:
out_queue= queues.TransferQueue()

class Flow(TreeFlow):

    def __init__(self):
        super().__init__()

        self.distributor = communicators.distributors.IteratorDistributor()
        self.compute = compute
        self.compute2 = compute2
        self.compute3 = ComputeClass(1, 2)
        self.printer1 = partials(print, 'step 1')
        self.printer2 = partials(print, 'step 2')

        self.chain(
            (None, 'distributor', queues.ThreadQueue()),
            ('distributor', 'compute', queues.ProcessQueue(5)),
            ('compute', 'compute2', queues.ProcessQueue(5)),
            ('compute', 'printer1', queues.TransferQueue()),
            ('compute2', 'printer2', queues.TransferQueue()),
            ('compute', 'compute3', queues.ProcessQueue()),
            ('compute3', None, out_queue),
            ('compute3', 'compute2'),
        )

flow = Flow()
flow

*-[ThreadQueue(name=None, runner=1)]

  |-(distributor:IteratorDistributor)

    |-[ProcessQueue(name=None, runner=5)]

      |-(compute:<function compute at 0x7fd8ac5d36a0>)

        |-[ProcessQueue(name=None, runner=5)]

          |-(compute2:<function compute2 at 0x7fd8ac5d3740>)

            |-[TransferQueue(name=None, runner=1)]

              |-(printer2:print(step 2,*,**))

        |-[TransferQueue(name=None, runner=1)]

          |-(printer1:print(step 1,*,**))

        |-[ProcessQueue(name=None, runner=1)]

          |-(compute3:ComputeClass)

            |-[TransferQueue((name=None, runner=1)]-*

            |-[ProcessQueue(name=None, runner=5)]

              |-(compute2:<function compute2 at 0x7fd8ac5d3740>)...

In [5]:
with flow:
   for q in flow.inputs.values():
      for i in range(10):
         q.put([i] * 2)

step 1 0
step 1 1
step 1 0
step 1 1
step 1 2
step 1 2
step 1 3
step 2 0c
step 1 3
step 1 4
step 1 4
step 2 1c
step 2 0c
step 2 1c
step 2 2c
step 1step 2 0dc
 5
step 1 5
step 2 1dc
step 1 6
step 1 6
step 1 7
step 2 0dc
step 2 1dc
step 2 2dc
step 1step 2 2c
 7
step 1 8
step 2 2dc
step 1 8
step 1 9
step 2 3c
step 1 9
step 2 3dc
step 2 3dc
step 2 3c
step 2 4c
step 2 4dc
step 2 4c
step 2 4dc
step 2 5c
step 2 5c
step 2 6c
step 2 5dc
step 2 5dc
step 2 6dc
step 2 6c
step 2 6dc
step 2 7c
step 2 7dc
step 2 8c
step 2 7c
step 2 7dc
step 2 8dc
step 2 8c
step 2 9c
step 2 8dc
step 2 9dc
step 2 9c
step 2 9dc


In [6]:
flow

*-[ThreadQueue(name=Identity, runner=1)]

  |-(distributor:IteratorDistributor)

    |-[ProcessQueue(name=compute, runner=5)]

      |-(compute:<function compute at 0x7fd8ac5d36a0>)

        |-[ProcessQueue(name=compute2, runner=5)]

          |-(compute2:<function compute2 at 0x7fd8ac5d3740>)

            |-[TransferQueue(name=partials, runner=1)]

              |-(printer2:print(step 2,*,**))

        |-[TransferQueue(name=partials, runner=1)]

          |-(printer1:print(step 1,*,**))

        |-[ProcessQueue(name=ComputeClass, runner=1)]

          |-(compute3:ComputeClass)

            |-[TransferQueue((name=None, runner=1)]-*

            |-[ProcessQueue(name=compute2, runner=5)]

              |-(compute2:<function compute2 at 0x7fd8ac5d3740>)...